In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
import numpy as np
import pandas as pd
import gc

In [3]:
ncpu = 8
rad = 0.004
ss = "k400"

In [4]:
path = "../Zillow2/Data/"
print("Read the properties and merge with coord")
coord = pd.read_csv(path + 'renamed_properties_2016.csv', usecols = ["id_parcel", "latitude", "longitude"])
print "Size of the properties data frame: ", coord.shape

Read the properties and merge with coord
Size of the properties data frame:  (2985217, 3)


In [5]:
features = ["id_parcel", "area_lot", "tax_property", "build_year", "area_total_calc", "tax_building", "tax_total"]

prop = pd.read_csv(path + 'renamed_properties_2016.csv', usecols = features)
print "Size of the properties data frame: ", prop.shape

Size of the properties data frame:  (2985217, 7)


In [6]:
index = pd.isnull(coord["latitude"])
coord = coord[~index]
prop = prop[~index]
print "Size of the properties data frame: ", coord.shape
print "Size of the properties data frame: ", prop.shape

Size of the properties data frame:  (2973780, 3)
Size of the properties data frame:  (2973780, 7)


In [7]:
coord.latitude = coord.latitude / 1e6
coord.longitude = coord.longitude / 1e6

In [ ]:
from sklearn.neighbors import NearestNeighbors
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()
print "number of cores " + str(num_cores)

In [ ]:
def splitDataFrameIntoSmaller(df, chunkSize = 100000): 
    listOfDf = list()
    numberChunks = len(df) // chunkSize + 1
    for i in range(numberChunks):
        listOfDf.append(i*chunkSize)
    listOfDf.append(len(df))
    return listOfDf

split_index = splitDataFrameIntoSmaller(prop, chunkSize=coord.shape[0]/num_cores)
print split_index

In [ ]:
def knn_avg(n1, n2, cc):
    avg_list = []
    for ii in range(n1, n2):
        index = np.setdiff1d(indices[ii], ii)
        if len(index) > 5:
            avg = prop.loc[index, cc].mean()
        else:
            avg = -1
        avg_list.append(avg)
    return avg_list

In [ ]:
features = ["area_lot", "tax_property", "build_year", "area_total_calc", "tax_building", "tax_total"]
feat_avg = ["area_lot_avg", "tax_property_avg", "build_year_avg", "area_total_calc_avg", 
            "tax_building_avg", "tax_total_avg"]

In [ ]:
knn = NearestNeighbors(n_neighbors=10000, algorithm='ball_tree', 
                        n_jobs = num_cores).fit(coord.loc[:, ["latitude", "longitude"]])

In [2]:
for rad in [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01]:
    ss = "k" + str(int(rad*1000))
    print "Folder " + ss

    indices = knn.radius_neighbors(coord.loc[:, ["latitude", "longitude"]], radius = rad,
                                          return_distance = False)
    ll = []
    for ii in indices:
        ll.append(len(ii))
    prop[ss] = ll
    
    for f1, f2 in zip(features, feat_avg):
        multi = Parallel(n_jobs=num_cores)(delayed(knn_avg)(split_index[i], split_index[i+1], f1)
                                   for i in range(0,len(split_index)-1))
    
        single = [x for xx in multi for x in xx]
    
        ss2 = f2 #ss + "_" + f2
        prop[ss2] = single
        print ss, f2
        
    name = ss + ".csv"
    prop[["id_parcel", ss] + feat_avg].to_csv(name, index=False)

k1
k2
k3
k4
k5
k6
k7
k8
k9
k10
